This script prints 10 frames for all the events of a test video.

The frames are saved as gifs in the `gifs/` folder.

WARNING: all frames go to RAM and can overload it. Choose wisely the number of events you want to visualize. Default is 5 (i.e., 5 events x 10 frames = 50 frames). 

In [ ]:
import os
import re
import pandas as pd
import cv2
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
import imageio

# Choose an event file (.csv)
session_path = '../data/datasets/'                                
csv_file = 'combined_events_level-1_clean_outset_vid.csv'  # mother of all TSV files - level 1 cleaned 
csv_file_path = os.path.join(session_path, csv_file)

# Choose a video file (.mp4)
video_file_path = '../output/videos/videos_full/sub-01_ses-003_task-shinobi_run-01_level-1_rep-01.mp4'

# Extract the base name of the video file (without path and extension)
video_file_name = os.path.basename(video_file_path)
video_file_name_no_ext = os.path.splitext(video_file_name)[0]

# Parse the video file name to extract subject, session, run, level, and repetition
pattern = r'(?P<subject>sub-[^_]+)_(?P<session>ses-[^_]+)_task-[^_]+_(?P<run>run-[^_]+)_(?P<level>level-[^_]+)_(?P<repetition>rep-[^_]+)'
match = re.match(pattern, video_file_name_no_ext)

if not match:
    raise ValueError("Video file name does not match the expected pattern")

subject = match.group('subject')
session = match.group('session')
run = match.group('run')
level = match.group('level')
repetition = match.group('repetition')

# Print extracted values to verify
print(f"Subject: {subject}")
print(f"Session: {session}")
print(f"Run: {run}")
print(f"Level: {level}")
print(f"Repetition: {repetition}")

# Load the event data
df = pd.read_csv(csv_file_path)

# Filter the DataFrame for the specified session, run, and repetition
df_filtered = df[(df['level'] == level) &
                 (df['session'] == session) & 
                 (df['run'] == run) & 
                 (df['repetition'] == repetition)]

# Load the video file with moviepy
video = VideoFileClip(video_file_path)

# Get frames per second (fps) from the video
fps = video.fps
print(f"Processing {video_file_path} with FPS: {fps}")

# Load the video file again to get frames for plotting
cap = cv2.VideoCapture(video_file_path)

# Get frames from the video file
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

# Release the video capture object
cap.release()

# Convert onset times to seconds and milliseconds
def convert_seconds_to_time_str(seconds):
    milliseconds = int((seconds % 1) * 1000)
    seconds = int(seconds)
    return f"{seconds}.{milliseconds:03d}"


# Create a dataframe where trial_type is not 'frame'
df_event = df_filtered[df_filtered.trial_type != 'frame']

# Extract onsets and trial types
onsets = df_event['onset_vid'].values
trial_types = df_event['trial_type'].values

# Counter to limit iterations ################################# 
counter = 0
max_iterations = 5

# Iterate over each onset
for onset, trial_type in zip(onsets, trial_types):
    onset_str = convert_seconds_to_time_str(onset)
    print(f'Onset (s): {onset_str}, Event: {trial_type}')

    # Get the index of the onset in the dataframe
    idx = df[df['onset_vid'] == onset].index[0]

    # Get the current onset and the next 10 onsets
    following_onsets = df.iloc[idx:idx+10]['onset_vid'].values

    # Get the frame indices for these onsets
    frames_indices = df.iloc[idx:idx+10]['frame'].values

    gif_frames = []
    for i, (onset_value, frame_index) in enumerate(zip(following_onsets, frames_indices)):
        onset_str = convert_seconds_to_time_str(onset_value)
    
        # Convert frame index to integer
        frame_index = int(frame_index)
    
        # Plot the frame
        frame_rgb = cv2.cvtColor(frames[frame_index], cv2.COLOR_BGR2RGB)
        gif_frames.append(frame_rgb)
        
        plt.imshow(frame_rgb)
        plt.title(f'Frame (Frame #{frame_index}, Onset {onset_str}s)')
        plt.xticks([])
        plt.yticks([])
        plt.show()
    
    # Save the frames as a GIF
    gif_output_path = f'../output/gifs/{subject}_{session}_{run}_{level}_{repetition}_{trial_type}_{counter}.gif'
    imageio.mimsave(gif_output_path, gif_frames, fps=fps, loop=0)
    print(f'Saved GIF to {gif_output_path}')

    # Increment the counter and check if the limit is reached ###########################################
    counter += 1
    if counter >= max_iterations:
        break